In [14]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import unicodedata
import json
import os



In [3]:
# Set headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"
}

- 30/35 responses per page, popular reviews section double counts the top 5 people
- 1.5-2s per response

In [21]:
# failed_fetch = 0
# usernames = set()
# for i in range(1):
#     url = "https://letterboxd.com/members/popular/this/year/page/{}/".format(i)

#     # Fetch the webpage
#     response = requests.get(url, headers=headers)
#     # Check if request was successful
#     if response.status_code == 200:
#         # Parse HTML
#         soup = BeautifulSoup(response.text, "html.parser")
#         user_elements = soup.select(".person-summary .name")

#         # Extract and print usernames
#         for user in user_elements:
#             usernames.add(user["href"][1:-1])
#     else:
#         print("failed iter {}", i)
#         failed_fetch += 1

## Scrape review data
- scraping 1 users reviews took roughly 1 min
- url doesnt work past 256

In [18]:
def save_review_to_json(file_path, user, film_title, unix_timestamp, rating, liked, review_text):
    # Create a dictionary with the extracted details
    review_data = {
        "user": user,
        "title": film_title,
        "review_date": unix_timestamp,
        "rating": rating,
        "liked": liked,
        "review_text": review_text
    }
    
    # Load existing data if the file exists, else start with an empty list
    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            try:
                data = json.load(file)
            except json.JSONDecodeError:
                data = []  # If the file is empty or corrupted, reset to an empty list
    else:
        data = []
    
    # Append new data to the list
    data.append(review_data)

    # Save the updated data back to the file
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)

In [20]:
for user in usernames:
    url = "https://letterboxd.com/{}/films/reviews/".format(user)
    
    # Fetch the webpage
    response = requests.get(url, headers=headers)
    # Check if request was successful
    if response.status_code == 200:
        # Parse HTML
        soup = BeautifulSoup(response.text, "html.parser")
        last_page = max(int(a.get_text()) for a in soup.select(".paginate-page a") if a.get_text().isdigit())

        for i in range(1, last_page + 1):
            url = "https://letterboxd.com/{}/films/reviews/page/{}".format(user, i)
            response = requests.get(url, headers=headers)
            soup = BeautifulSoup(response.text, "html.parser")

            # Find all film detail blocks
            film_blocks = soup.find_all("div", class_="film-detail-content")
            # print(user, url)
            # Iterate over each block and extract the required information
            for film in film_blocks:
                # Extract film title
                film_title_tag = film.find("h2", class_="headline-2 prettify").find("a")
                film_title = film_title_tag.text.strip() if film_title_tag else "Unknown Title"
                # print(film_title)


                # Extract star rating
                rating_tag = film.find("span", class_="rating")
                rating_text = rating_tag.text.strip() if rating_tag else None
                if rating_text:
                    rating = rating_text.count('\u00BD')*0.5 + rating_text.count('\u2605')

                # Extract date
                date_tag = film.find("span", class_="_nobr")
                date_string = date_tag.text.strip()
                date_object = datetime.strptime(date_string, "%d %b %Y")
                unix_timestamp = int(date_object.timestamp())

                # Check if "icon-liked" exists in the attribution block
                liked_icon = film.find("span", class_="has-icon icon-16 icon-liked")
                liked = 1 if liked_icon else 0

                # Extract review text
                review_body_tag = film.find("div", class_="body-text -prose js-review-body js-collapsible-text")
                review_text = review_body_tag.get_text(" ", strip=True) if review_body_tag else None

                # save review to json
                save_review_to_json("../letterboxd_proj_data/test_reviews.json", user, film_title, unix_timestamp, rating, liked, review_text)
    else:
        print("failed iter {}", i)
        failed_fetch += 1
    break